In [68]:
import pandas as pd

# 替换为您的Excel文件路径
file_path = 'Viral_RNA_dataset_v1.xlsx'

df = pd.read_excel(file_path)

# 初始化一个空的DataFrame来存储处理后的数据
new_df = pd.DataFrame()

# 记录上一个非空Entry_ID的索引和ID
last_valid_index = None
last_valid_id = None

# 存储已合并行的ID
merged_ids = []

# 遍历DataFrame中的每一行
for index, row in df.iterrows():
    if pd.isnull(row['Entry_ID']):
        # 如果Entry_ID为空，则将信息添加到上一行的相应列中
        if last_valid_index is not None:
            for col in df.columns:
                # 检查目标单元格是否为空，并且当前行该列有值
                if pd.isnull(new_df.at[last_valid_index, col]) and not pd.isnull(row[col]):
                    new_df.at[last_valid_index, col] = row[col]
                # 对于字符串类型的列，确保即使原始数据不是字符串，也能安全拼接
                elif not pd.isnull(row[col]):
                    # 将新的值转换为字符串并追加到原有数据后面
                    new_value = str(row[col]).strip()
                    original_value = str(new_df.at[last_valid_index, col])
                    new_df.at[last_valid_index, col] = original_value + ' ' + new_value if original_value else new_value
            # 记录下合并到的行的ID
            merged_ids.append(last_valid_id)
    else:
        # 如果Entry_ID不为空，直接将行添加到new_df中，并更新最近的非空Entry_ID的索引和ID
        new_df = pd.concat([new_df, pd.DataFrame([row])], ignore_index=True)
        last_valid_index = new_df.last_valid_index()
        last_valid_id = row['Entry_ID']

# 输出所有合并到的行的ID
print("Merged to row IDs:", merged_ids)

# 处理合并单元格后的NaN值
new_df.fillna('', inplace=True)

# 数据清洗
new_df['pKd'] = new_df['pKd'].apply(lambda x: float(str(x).replace(" ", "").split('\n')[0]) if x != '' else None)
new_df['Target_RNA_sequence'] = new_df['Target_RNA_sequence'].apply(lambda x: str(x).replace(" ", ""))
new_df['SMILES'] = new_df['SMILES'].apply(lambda x: str(x).replace(" ", ""))

# 保存清洗后的DataFrame到新的Excel文件
output_file_path = 'cleaned/Viral_RNA.xlsx'  # 确保这个路径是存在的，或者自动创建目录
new_df.to_excel(output_file_path, index=False)


Merged to row IDs: [238.0, 272.0, 711.0, 734.0, 854.0, 867.0, 953.0, 966.0, 994.0, 1002.0, 1130.0, 1143.0, 1155.0, 1782.0, 2054.0, 2147.0, 2190.0, 2383.0, 2391.0, 2480.0]


In [59]:
import pandas as pd

# 替换为您的Excel文件路径
file_path = 'Riboswitch_dataset_v1.xlsx'

# 读取Excel文件
df = pd.read_excel(file_path)

# 初始化一个新的DataFrame来存储处理后的数据
new_df = pd.DataFrame()

# 记录需要删除的行索引
rows_to_delete = []

# 记录上一个非空Entry_ID的索引
last_valid_index = -1

# 存储合并信息的字典 {非空Entry_ID: [被合并的行索引]}
merged_info = {}

# 遍历DataFrame中的每一行
for index, row in df.iterrows():
    if pd.isnull(row['Entry_ID']):
        # 如果Entry_ID为空，收集该行索引以备后续删除
        rows_to_delete.append(index)
        if last_valid_index != -1:
            # 将当前行的信息合并到上一个非空Entry_ID的行中
            for col in df.columns:
                if isinstance(row[col], str) and pd.notnull(row[col]):
                    # 对字符串列，进行内容追加
                    new_df.at[last_valid_index, col] += ' ' + row[col].strip()
                elif pd.isnull(new_df.at[last_valid_index, col]) and pd.notnull(row[col]):
                    # 对于其他类型的列，如果新的DataFrame中对应位置是空，则直接赋值
                    new_df.at[last_valid_index, col] = row[col]
            # 记录合并信息
            valid_id = new_df.at[last_valid_index, 'Entry_ID']
            if valid_id not in merged_info:
                merged_info[valid_id] = [index]
            else:
                merged_info[valid_id].append(index)
    else:
        # 如果Entry_ID不为空，将行添加到新的DataFrame中
        new_df = pd.concat([new_df, pd.DataFrame([row]).reset_index(drop=True)], ignore_index=True)
        last_valid_index = new_df.last_valid_index()

# 输出每个非空ID行合并了哪些行
for valid_id, indexes in merged_info.items():
    print(f"Entry_ID {valid_id} merged rows: {indexes}")
new_df['pKd'] = new_df['pKd'].apply(lambda x: float(str(x).replace(" ", "").split('\n')[0]))
new_df['Target_RNA_sequence'] = new_df['Target_RNA_sequence'].apply(lambda x: (str(x).replace(" ", "")))
new_df['SMILES'] = new_df['SMILES'].apply(lambda x: (str(x).replace(" ", "")))

output_file_path = 'cleaned/Riboswitch.xlsx'  # 指定新的文件路径和文件名
new_df.to_excel(output_file_path, index=False)

TypeError: unsupported operand type(s) for +=: 'float' and 'str'

In [43]:
import pandas as pd
#检查异常的pkd
# 文件路径
input_path = 'cleaned/miRNA.xlsx'

# 读取Excel文件
df = pd.read_excel(input_path)
df['pKd'] = df['pKd'].apply(lambda x: float(str(x).replace(" ", "").split('\n')[0]))

# 查找pKd列值大于10的所有行
high_pkd_entries = df[df['pKd'] > 10]

# 输出这些行的Entry_ID
entry_ids = high_pkd_entries['Entry_ID'].tolist()
print("Entry_IDs with pKd > 10:", entry_ids)


Entry_IDs with pKd > 10: []


In [18]:
import pandas as pd
#检查是否有重复ID
# 路径可能需要根据您的文件系统进行修改
file_path = 'Viral_RNA_dataset_v1.xlsx'

# 读取Excel文件
df = pd.read_excel(file_path)

# 初始化空列表来存储重复的Entry_ID
duplicate_entry_ids = []

# 遍历DataFrame的行
for i in range(1, len(df)):  # 从第二行开始，因为我们会和前一行比较
    # 比较当前行的Entry_ID与前一行的Entry_ID
    if df.loc[i, 'Entry_ID'] == df.loc[i-1, 'Entry_ID']:
        # 如果它们相同，将该ID添加到列表中
        duplicate_entry_ids.append(df.loc[i, 'Entry_ID'])

# 输出重复的Entry_ID
for entry_id in duplicate_entry_ids:
    print(entry_id)

255.0
734.0
867.0
